In [1]:
from json import dumps
def safeVisNetworkJSONToHTMLFile(jsonData, htmlFilePath):
    with open(htmlFilePath, "w") as htmlFile:
        htmlFile.write(
            f"""
            <!DOCTYPE html>
            <html lang="en-US">
            
            <head>
                <meta charset="UTF-8">
                <meta name="viewport" content="width=device-width, initial-scale=1">
                <title>Inline vis</title>
            </head>
            
            <body>
                <div id="mynetwork" style="width:100vw; height:100vh;"></div>
                <script src=" https://cdn.jsdelivr.net/npm/vis-data@7.1.9/peer/umd/vis-data.min.js "></script>
                <script src=" https://cdn.jsdelivr.net/npm/vis-network@9.1.9/peer/umd/vis-network.min.js "></script>
                <link href=" https://cdn.jsdelivr.net/npm/vis-network@9.1.9/styles/vis-network.min.css " rel="stylesheet">
                <script>
                    var jsonData = {dumps(jsonData)};
                    // create a network
                    var container = document.getElementById("mynetwork");
                    var options = {{}};
                    var network = new vis.Network(container, jsonData, options);
                </script>
            </body>
            
            </html>
            """
        )

In [28]:

from urllib.parse import quote
import latextools
import drawsvg as draw
latex_eq = latextools.render_snippet(
    r'$\hbar \omega$',#r'$\sqrt{\sum_{k = 0}^{N}\binom{N}{k}p^k(1-p)^{N - k}(k - N_+)^{2}}$',
    commands=[latextools.cmd.all_math])
svg_eq = latex_eq.as_svg()

print(type(svg_eq))

# Use the rendered latex in a vector drawing
svgwidth = 300
svgheight = 100
d = draw.Drawing(svgwidth, svgheight, origin='center', displayInline=False)
d.append(draw.Ellipse(0, 0, svgwidth/2 - 1, svgheight/2 - 1, fill='yellow', stroke='black', stroke_width=2))
d.draw(svg_eq, x=0, y=0, center=True, scale=0.9*min(svgwidth/svg_eq.width, svgheight/svg_eq.height))


simpleSvg = d.as_svg()
svgUrl = f"data:image/svg+xml,{quote(simpleSvg)}"
safeVisNetworkJSONToHTMLFile(
    {
        "nodes" : [
            {"id": 1, "label": 'Bla bla bla \nbla bla und so weiter', "group": 'United States', "shape" : "box"},
            {"id": 2, "label": 'Barak Osman', "group": 'United States'},
            {"id": 3, "label": 'Miley Cyrus', "shape" : "image", "image" : svgUrl},
            {"id": 4, "label": 'Pope Francis', "group": 'Vatican'},
            {"id": 5, "label": 'Vladimir Putin', "group": 'Rusia'}
        ],
        "edges" : [
            {"from": 1, "to": 2, "arrows" : "to"},
            {"from": 1, "to": 3},
            {"from": 3, "to": 4},
            {"from": 3, "to": 5}
        ]
    }, "output.html"
)

<class 'latextools.convert.Svg'>


In [45]:
def convertFormulaToSVGURL(formula):
    latex_eq = latextools.render_snippet(
        "$" + formula + "$",
        commands=[latextools.cmd.all_math])
    svg_eq = latex_eq.as_svg()
    svgwidth = 300
    svgheight = 100
    d = draw.Drawing(svgwidth, svgheight, origin='center', displayInline=False)
    d.append(draw.Ellipse(0, 0, svgwidth/2 - 1, svgheight/2 - 1, fill='yellow', stroke='black', stroke_width=2))
    d.draw(svg_eq, x=0, y=0, center=True, scale=0.8*min(svgwidth/svg_eq.width, svgheight/svg_eq.height))
    simpleSvg = d.as_svg()
    return f"data:image/svg+xml,{quote(simpleSvg)}"

In [40]:
def convertFormulaSymbolDefinitionToSVGURL(symbol):
    latex_eq = latextools.render_snippet(
        "where $" + symbol + "$ is",
        commands=[latextools.cmd.all_math])
    svg_eq = latex_eq.as_svg()
    # Fit the symbol definition into a rectangle
    d = draw.Drawing(svg_eq.width + 6, svg_eq.height + 6, origin='center', displayInline=False)
    d.append(draw.Rectangle(-d.width/2, -d.height/2, d.width, d.height, fill='lightgreen', stroke='black', stroke_width=2))
    d.draw(svg_eq, x=0, y=0, center=True)
    simpleSvg = d.as_svg()
    return f"data:image/svg+xml,{quote(simpleSvg)}"

In [50]:
def convertFormularNetworkToVisNetworkJSON(formularNetwork):
    """
    FormularNetwork is a list of triples (subj, pred, obj).
    Subj and obj are eather formulas ("f: ... ") or text ("t: ...").
    The pred is one of the following relations:
    - "is defined by" ("!")
    - "equals" ("=")
    - "where ??? is" (": ...")
    """
    nodes = []
    edges = []
    indexPerTerm = {}
    indexCounter = 0
    for subj, pred, obj in formularNetwork:
        for term in subj, obj:
            if term not in indexPerTerm:
                indexCounter += 1
                indexPerTerm[term] = indexCounter
                if term.startswith("f: "):
                    formulaURL = convertFormulaToSVGURL(term[3:])
                    nodes.append({
                        "id": indexCounter, 
                        "shape": "image",
                        "image": formulaURL
                    })
                elif term.startswith("t: "):
                    nodes.append({
                        "id": indexCounter, 
                        "label": term[3:], 
                        "font": {"size": 20}, 
                        "color" : {"background": "rgba(140, 220, 255, 255)", 
                                    "border": "rgba(0, 0, 255, 255)"}})
                else:
                    raise ValueError(f"Unknown term type: {term}")
        indexCounter += 1
        if pred == "!":
            nodes.append({
                "id": indexCounter, 
                "label": "is defined by", 
                "font": {"size": 16},
                "color" : {"background": "rgba(255, 200, 100, 255)", 
                            "border": "darkorange"},
                "shape" : "box"})
            edges.append({"from": indexPerTerm[subj], "to": indexCounter, "color": "rgba(200, 100, 60, 255)", "width": 3})
            edges.append({"from": indexCounter, "to": indexPerTerm[obj], "arrows": "to", "color": "rgba(200, 100, 60, 255)", "width": 3})
        elif pred == "=":
            nodes.append({
                "id": indexCounter, 
                "label": "equals", 
                "font": {"size": 16},
                "color" : {"background": "rgba(255, 100, 100, 255)", 
                            "border": "darkred"},
                "shape" : "box"})
            edges.append({"from": indexCounter, "to": indexPerTerm[subj], "color": "rgba(200, 60, 60, 255)", "width": 3, "arrows": "to"})
            edges.append({"from": indexCounter, "to": indexPerTerm[obj], "color": "rgba(200, 60, 60, 255)", "width": 3, "arrows": "to"})
        elif pred.startswith(": "):
            formulaURL = convertFormulaSymbolDefinitionToSVGURL(pred[2:])
            nodes.append({
                "id": indexCounter, 
                "shape": "image",
                "size": 11,
                "image": formulaURL})
            edges.append({"from": indexPerTerm[subj], "to": indexCounter, "color": "rgba(100, 200, 60, 255)", "width": 3})
            edges.append({"from": indexCounter, "to": indexPerTerm[obj], "arrows": "to", "color": "rgba(100, 200, 60, 255)", "width": 3})
    return {"nodes": nodes, "edges": edges}

In [51]:
h_w = r'f: \hbar \omega'
hv = r'f: h f'
E = r'f: E'
energy = "t: Energy of the Photon"
formulaNetwork = [
    [energy, "!", h_w],
    [h_w, "=", hv],
    [energy, "!", hv],
    [hv, "=", E],
    [E, ": E", energy],
    [hv, ": h", "t: Planck's constant"],
    [hv, ": f", "t: Frequency of the photon"],
    [h_w, ": \hbar", "t: Reduced Planck's constant"],
    [h_w, ": \omega", "t: Angular frequency of the photon"]
    ]
safeVisNetworkJSONToHTMLFile(convertFormularNetworkToVisNetworkJSON(formulaNetwork), "output.html")